In [ ]:
import pandas as pd
from openai import AzureOpenAI
from datetime import date
import my_secrets as sc

In [ ]:
# Configuration   TODO move to config and secrets

# OpenAI API variables
AZURE_OPENAI_API_KEY = "9ztHj1sVgWA0qLXL7KOM7URgYaMizodRHuWijPPGLO4J7Bi0cSExJQQJ99AKAC5RqLJXJ3w3AAABACOGKURw" 
AZURE_OPENAI_API_VERSION = "2024-02-15-preview"
AZURE_OPENAI_ENDPOINT = sc.AZURE_OPENAI_ENDPOINT

In [ ]:
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
)

# Function to prompt the LLM
def prompt_gpt(prompt, context=None, system=None):

    conversation = []

    if system:
        conversation.append({"role": "system", "content": system})

    if context:
        conversation.append({"role": "system", "content": context})

    conversation.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
                    model="gpt-4o",
                    messages=conversation,
                    temperature=0.3,
                    # max_tokens=200,
                    top_p=0.95,
                    frequency_penalty=0,
                    presence_penalty=0,
                    stop=None,
    )

    finish_reason = response.choices[0].finish_reason
    if finish_reason != "stop":
        print(finish_reason)

    return response.choices[0].message.content

In [ ]:
#context = " Ik wilde dus naar een vriendin van mij gaan met Kerst. Zij zegt tegen mij met al die meuk die ik mee moet, bak je beter gewoon even de auto. Ik heb van die bezoekers dingen, daar aan is niet zo sterk stuur. Dus ik geef haar mijn kenteken, maar ik ben moe weet je. Dus ik geef een z-door in plaats van niks. Die heeft wel allemaal kenteken zitten. Zij vond het keurig netjes in, maar ja, wat fout dus. Maar dat wist ik toen nog niet. Paar weken later, een dikke boete op de mat. Die wil ik natuurlijk niet betalen."
#context = "Het is echt ongelofelijk. Ik had in de keringen eindelijk mijn Fiat Panda tussen twee Teslas gewurmd. Ga ik nu de automaat om 6 euro per uur weg te pissen, zie ik zo'n scanauto langsrijden. Ik zei tegen mijn vrienden, het gaat niet goed hoor. Die ziet ons staan, maar we zijn nog met onze passie bezig. En inderdaad, onze plaat AB09CC stond er nog niet in. Want een paar weken later lag die bon op mijn mat."
context = " Hallo! Ik heb net een boete gekregen voor mijn parktier. Ik zie van een paar weken geleden. Maar er was daar in de Versterverte geen boer te bekennen. Dus het leek me nog ontrecht dat ik deze boete moet betalen. Ik stond daar op 14 mei om 16.37 met mijn mini clubman aan de vrijheid slaan. Dank u!"

In [ ]:
system = 'You are a helpful AI assistent which helps to make sense of a transcribed text in Dutch. The conversation is about a parking fine' 

In [ ]:
# Get 'reden van bezwaar'
analysis_prompt = 'Based on the text, can you describe in 1 sentence what the reason is that this person wants to not pay the fine'
response = prompt_gpt(prompt=analysis_prompt, context=context, system=system)
print(response)

In [ ]:
# Get 'reden van bezwaar' category
analysis_prompt = "Based on the text, please deduct what the reason is the person doesn't want to pay the fine. You can pick from: ['no money on bank account', 'filled in wrong license plate number', 'didn't know they have to pay parking costs', 'parking costs are too high', 'only parked very shortly to offload something', 'did not yet pay when check was performed']. Please only return the selected category, exactly as described here."
response = prompt_gpt(prompt=analysis_prompt, context=context, system=system)
print(response)

In [ ]:
# Get information to fill in form
analysis_prompt = "Based on the text, fill in the following form as a python dictionary: dict = {'naam':'name of person that needs to pay', 'adres': 'address of person that needs to pay the fine', 'datum': 'date of parking', 'aanslagnummer parkeerbon': 'the number indicated on the fine', 'kenteken': 'license plate number', 'reden van bezwaar': 'summary of reason why they don't want to pay the parking fine']. If the information is not available from the text, fill in np.nan.  Only return the dict =  and the dictionary itself, no other text or code."
response = prompt_gpt(prompt=analysis_prompt, context=context, system=system)
print(response)

In [ ]:
# Get information to fill in form and question for missing information
analysis_prompt = "Based on the text, fill in memory the following form as a python dictionary: dict = {'naam':'name of person that needs to pay', 'adres': 'address of person that needs to pay the fine', 'datum': 'date of parking', 'aanslagnummer parkeerbon': 'the number indicated on the fine', 'kenteken': 'license plate number', 'reden van bezwaar': 'summary of reason why they don't want to pay the parking fine']. If the information is not available from the text, fill in np.nan.  Return the dictionary and a question (in Dutch) requesting the information for where you filled in np.nan "
response = prompt_gpt(prompt=analysis_prompt, context=context, system=system)
print(response)